In [31]:
import json  
import os, sys, pathlib
import pickle

import pandas as pd
import matplotlib.pyplot as plt
import cv2

import importlib

BASE_DIR = pathlib.Path.cwd().parent


In [3]:
sys.path.append(str(BASE_DIR / 'src'))

import ollama_manager as om
import main
import STAR_utils.visualization_tools.qa_visualization as qaviz


In [5]:
# please replace following data path to your local path
# all data can be download from our homepages (http://star.csail.mit.edu or https://bobbywu.com/STAR)

# root_dir = '/root/user/'
raw_video_dir = BASE_DIR / 'data/datasets/action-genome/Charades_v1_480/'
raw_frame_dir = BASE_DIR / 'data/datasets/action-genome/frames/'
annotation_dir = BASE_DIR / 'src/STAR_utils/annotations/STAR_classes/'
# pose_dir = root_dir + 'STAR/pose/'
# qa_train_dir = BASE_DIR /'data/datasets/STAR/STAR_annotations/STAR_train.json' 
qa_val_data = BASE_DIR / 'data/datasets/STAR/STAR_annotations/STAR_val.json' 
# qa_test_dir = BASE_DIR / 'data/datasets/STAR/STAR_annotations/STAR_test.json' 
save_video_dir = BASE_DIR / 'visualization_tmp'
fps = pickle.load(open(annotation_dir / 'video_fps','rb'))


In [6]:
star_data = []
with open(qa_val_data, 'r') as f:
    star_data = json.load(f)


In [ ]:
star_df = pd.DataFrame(star_data)
star_df = star_df.set_index('question_id')
star_df


,question,video_id,start,end,answer,question_program,choices,situations
question_id,,,,,,,,
Interaction_T1_13,Which object was tidied up by the person?,6H78U,11.1,19.6,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The closet/cabine...","{'000206': {'rel_pairs': [['o000', 'o027'], ['..."
Interaction_T1_14,Which object was tidied up by the person?,6H78U,15.6,22.7,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The blanket.', 'c...","{'000289': {'rel_pairs': [['o000', 'o027'], ['..."
Interaction_T1_31,Which object was thrown by the person?,RNLTR,7.3,13.5,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The pillow.', 'ch...","{'000132': {'rel_pairs': [['o000', 'o019'], ['..."
Interaction_T1_32,Which object was put down by the person?,RNLTR,11.6,16.4,The shoe.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The food.', 'choi...","{'000202': {'rel_pairs': [['o000', 'o019'], ['..."
Interaction_T1_40,Which object was tidied up by the person?,VNQTH,2.7,9.2,The closet/cabinet.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The broom.', 'cho...","{'000110': {'rel_pairs': [['o000', 'o014'], ['..."
...,...,...,...,...,...,...,...,...
Feasibility_T6_1453,What is the person able to do after taking the...,L9ANI,7.3,12.9,Wash the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Wash the table.',...","{'000227': {'rel_pairs': [['o000', 'o006'], ['..."
Feasibility_T6_1454,What is the person able to do after walking th...,L9ANI,0.0,4.1,Take the towel.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Take the towel.',...","{'000033': {'rel_pairs': [['o000', 'o006']], '..."
Feasibility_T6_1455,What is the person able to do after walking th...,L9ANI,0.0,4.1,Wash the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Throw the bag.', ...","{'000033': {'rel_pairs': [['o000', 'o006']], '..."


In [30]:
sorted(star_df.loc['Interaction_T1_14']['situations'].keys())


['000286',
 '000289',
 '000294',
 '000301',
 '000303',
 '000308',
 '000313',
 '000314',
 '000316',
 '000318',
 '000320',
 '000322',
 '000331',
 '000332',
 '000336',
 '000337',
 '000339',
 '000364',
 '000370',
 '000377',
 '000388',
 '000389',
 '000392']

In [8]:
star_data[0].keys()


dict_keys(['question_id', 'question', 'video_id', 'start', 'end', 'answer', 'question_program', 'choices', 'situations'])

In [13]:
ids = [d['question_id'] for d in star_data]
len(ids), len(set(ids))


(7098, 7098)

In [17]:
ids[0]


'Interaction_T3_6051'

In [18]:
import random

half_size = len(ids) // 2
random.shuffle(ids)
first_half = ids[:half_size]
second_half = ids[half_size:]


In [19]:
first_half[0]


'Interaction_T2_3150'

In [20]:
with open('../src/val1_ids.txt', 'w') as f:
    for i in first_half:
        f.write(f"{i}\n")
        
with open('../src/val2_ids.txt', 'w') as f:
    for i in second_half:
        f.write(f"{i}\n")


In [50]:
a = PromptDataset(BASE_DIR / 'outputs/responses_genframes:7b_202504010_22:47:00.jsonl')


TypeError: PromptDataset.__init__() missing 1 required positional argument: 'prompt_formatter'

In [ ]:
sorted_positions = sorted(load_positions.items(), key=lambda x: x[1])
